# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena 人間による好み予測チャレンジ」に取り組んでいます。具体的には、大規模言語モデル（LLM）によって生成されたチャット応答の中から、ユーザーが好みそうな応答を予測するための機械学習モデルを構築しています。

主な手法としては、Natural Language Processing (NLP)を用いたテキスト分析が行われており、以下のようなライブラリや技術が使用されています：

- **NumPy**や**Pandas**：基本的なデータ処理を行うために用いられます。
- **NLTK**：テキストをトークン化したり、品詞タグ付け、頻度分布の計算などに使用されます。
- **TextStat**：可読性スコアの計算に利用されており、Flesch-KincaidグレードやGunning-Fogインデックスなど多様な指標を提供します。
- **TextBlob**：テキストの感情分析を実行します。
- **SpaCy**：テキスト処理（トークン化など）に使われ、受動態文のカウント等も行います。
- **scikit-learn**：Gradient BoostingやXGBoost、LightGBM、CatBoostなどの強化学習アルゴリズムを使用してモデルを訓練し、性能評価には対数損失（log loss）が採用されています。
- **Optuna**：ハイパーパラメータの最適化に利用されており、異なるモデルタイプ（XGBClassifier, LGBMClassifier, CatBoostClassifier）に対して最適なパラメータを選定します。
- **Concurrent Futures**：テキスト統計を並列処理するために使用され、計算効率が向上しています。

具体的な流れとしては、まずトレーニングデータとテストデータを読み込み、各テキストに対して可読性スコアや感情スコア、文の統計を計算し、特徴量を生成します。次に、これらの特徴量を用いてモデルを訓練し、最終的にテストデータに対する予測を行い、結果をCSVファイルとして保存します。

全体的には、ユーザーの好みを正確に予測するための多様なテキスト分析手法と機械学習アルゴリズムを組み合わせたアプローチが強調されています。

---


# 用語概説 
以下は、Jupyter Notebookの内容に関連する専門用語の簡単な解説です。特に初心者がつまずきやすい用語や、このコンペティション特有のドメイン知識に焦点を当てています。

### 専門用語の解説

1. **可読性スコア (Readability Scores)**:
   - テキストがどれだけ容易に理解できるかを定量化する指標。一般的には、英語の文章に対してどの年齢層の読者が理解可能かを示すために使用される。例として、Flesch-KincaidグレードやGunning-Fogインデックスがある。

2. **Flesch-Kincaidグレード (Flesch-Kincaid Grade)**:
   - 英語の文章の難易度を示す指標で、読者が理解するのに必要な学年を示す。数値が高いほど、文章が難しくなる。

3. **Gunning-Fogインデックス**:
   - テキストの可読性を測る指標で、文章中の難しい単語の割合と文の長さを考慮して、理解に必要な年数を示す。

4. **SMOGインデックス (SMOG Index)**:
   - ショート文の可読性を測定するための指数で、特に公共の場や教育のために作成されたもので、難解な単語の数に基づいて計算される。

5. **自動可読性インデックス (Automated Readability Index)**:
   - テキストの難易度を測定するための指標で、単語数と文の長さから計算される。数値が高いと難易度が高くなる。

6. **Coleman-Liau Index**:
   - テキストの可読性を示す指標で、文字数に基づいており、特に単純な計算で可読性を測定する方式を採用している。

7. **名詞句 (Noun Phrases)**:
   - 名詞とその修飾語から成る構造。文章中で主要な意味を構成する要素であり、その数を解析することでテキストの構造を理解できる。

8. **感情極性 (Sentiment Polarity)**:
   - テキストが持つ感情の方向性。一般的に、ポジティブ、ネガティブ、ニュートラルのどれかに分類される。テキストの感情的なトーンを評価する際に使われる。

9. **受動態 (Passive Voice)**:
   - 文の構造の一つで、行為によって影響を受ける側（目的語）が主語となる形式。例えば、「彼によって書かれた手紙」は受動態となる。

10. **POS (Part of Speech) タグ**:
    - 単語に対する品詞（名詞、動詞、形容詞など）のラベル。自然言語処理において、文の構造を理解するために重要な役割を果たす。

11. **並列処理 (Parallel Processing)**:
    - 複数のプロセスを同時に実行する方法。データ処理にかかる時間を短縮するために用いられる。

12. **Optuna**:
    - ハイパーパラメータ最適化のためのフレームワーク。経済的に効率的に最適なモデルのパラメータを探索することができる。

13. **ログ損失 (Log Loss)**:
    - モデルの予測性能を評価するための指標。予測確率と実際のラベルとの違いを計測し、数値が小さいほど良いモデルとされる。

これらの用語は、Jupyter Notebookの内容に特有のものであり、理解することで機械学習・深層学習の解析やモデルの構築に役立つでしょう。

---


In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これは、kaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、以下のいくつかの便利なパッケージを読み込むことができます

import numpy as np # 線形代数
import pandas as pd # データ処理、CSVファイルの入出力（例: pd.read_csv）

# 入力データファイルは読み取り専用の "../input/" ディレクトリにあります
# 例えば、これを実行すると（実行ボタンをクリックするかShift+Enterを押す）、入力ディレクトリ内のファイルがすべてリストされます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 現在のディレクトリ(/kaggle/working/)には最大20GBまで書き込むことができ、バージョンを作成する際に出力として保存されます
# 一時ファイルは/kaggle/temp/に書き込むことができますが、現在のセッションの外では保存されません

In [ ]:
import sys
sys.path.append('/kaggle/input/textstat-pypi/Pyphen-0.9.3-py2.py3-none-any.whl')
!pip install '/kaggle/input/textstat-pypi/Pyphen-0.9.3-py2.py3-none-any.whl'

In [ ]:
sys.path.append('/kaggle/input/textstat-pypi/textstat-0.7.0-py3-none-any.whl')
!pip install '/kaggle/input/textstat-pypi/textstat-0.7.0-py3-none-any.whl'

In [ ]:
sys.path.append('/kaggle/input/textstat-pypi/textstat-0.7.0-py3-none-any.whl')
!pip install '/kaggle/input/textstat-pypi/textstat-0.7.0-py3-none-any.whl'

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import textstat
from textblob import TextBlob
import spacy
import concurrent.futures
import optuna
from sklearn.metrics import log_loss
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
# nltk.download('punkt')  # NLTKのpunktモデルをダウンロードします

In [ ]:
def calculate_readability_scores(text):
    # この関数は、さまざまな可読性スコアを計算します
    return {
        "flesch_kincaid_grade": textstat.flesch_kincaid_grade(text),  # Flesch-Kincaid グレードを計算
        "gunning_fog": textstat.gunning_fog(text),  # Gunning-Fog インデックスを計算
        "smog_index": textstat.smog_index(text),  # SMOGインデックスを計算
        "ari": textstat.automated_readability_index(text),  # 自動可読性インデックスを計算
        "coleman_liau_index": textstat.coleman_liau_index(text)  # Coleman-Liau インデックスを計算
    }

def count_noun_phrases(text):
    blob = TextBlob(text)  # TextBlobオブジェクトを作成
    return len(blob.noun_phrases)  # 名詞句の数を返す

def analyze_sentiment(text):
    blob = TextBlob(text)  # TextBlobオブジェクトを作成
    return blob.sentiment.polarity  # テキストの感情極性を返す

def count_passive_voice(text):
    doc = nlp(text)  # SpaCyを使用してテキストを処理
    return sum(1 for token in doc if token.dep_ == 'auxpass')  # 受動態の文の数をカウント

def pos_tag_frequencies(text):
    words = word_tokenize(text)  # テキストを単語にトークン化
    tags = nltk.pos_tag(words)  # 各単語に品詞タグを付与
    freq_dist = nltk.FreqDist(tag for (word, tag) in tags)  # 品詞の頻度分布を計算
    # 一貫した辞書形式で頻度を格納することを保証
    return {tag: freq for tag, freq in freq_dist.items()}

def text_statistics(text):
    stats = calculate_readability_scores(text)  # 可読性スコアの計算
    stats.update({
        "word_count": len(word_tokenize(text)),  # 単語数を計算
        "char_count": len(text),  # 文字数を計算
        "sentence_count": len(sent_tokenize(text)),  # 文数を計算
        "avg_word_length": sum(len(word) for word in word_tokenize(text)) / len(word_tokenize(text)),  # 平均単語長を計算
        "avg_sentence_length": sum(len(sent) for sent in sent_tokenize(text)) / len(sent_tokenize(text)),  # 平均文長を計算
        "lexical_diversity": len(set(word_tokenize(text))) / len(word_tokenize(text)),  # 語彙の多様性を計算
        "noun_phrases_count": count_noun_phrases(text),  # 名詞句の数を計算
        "sentiment": analyze_sentiment(text),  # 感情分析を実行
        "passive_voice_count": count_passive_voice(text),  # 受動態のカウントを実行
    })
    # POSタグの頻度を主な統計辞書にマージ
    pos_tags = pos_tag_frequencies(text)
    for tag, count in pos_tags.items():
        stats[f'pos_tag_{tag}'] = count  # 各品詞の頻度を辞書に追加
    return stats

def parallel_apply(df, column):
    # NaN値をドロップしてエラーを回避
    texts = df[column].dropna()  # NaNを除外したテキストのリストを作成

    # ProcessPoolExecutorを使用して並列に関数を適用
    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = list(executor.map(text_statistics, texts))  # 並列処理でテキスト統計を計算

    # 辞書のリストをDataFrameに変換
    results_df = pd.DataFrame(results)

    # 欠損POSタグを0で埋め、データ型を適切に変換する処理を自動的に行う
    results_df.fillna(0, inplace=True)  # NaNを0で埋める
    for col in results_df.columns:
        if results_df[col].dtype == float:
            results_df[col] = results_df[col].astype(int)  # float型の列をint型に変換

    return results_df

In [ ]:
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # トレーニングデータを読み込む
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込む
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  # 提出サンプルを読み込む
print('データがインポートされました')  # データインポート完了のメッセージ

In [ ]:
train.shape  # トレーニングデータの形状を表示

In [ ]:
# def parallel_apply(df, column):
#     with concurrent.futures.ProcessPoolExecutor() as executor:
#         results = list(executor.map(text_statistics, df[column].dropna()))  # NaNを処理するためにdropnaを使用
#     return pd.DataFrame(results)  # DataFrameを返す関数（コメントアウト）

In [ ]:
# 'prompt'および'response'列に対してparallel_applyを適用
nlp = spacy.load('en_core_web_sm')  # SpaCyの英語モデルをロード
prompt_stats_df = parallel_apply(train, 'prompt')  # プロンプトの統計を計算
response_a_stats_df = parallel_apply(train, 'response_a')  # レスポンスAの統計を計算
response_b_stats_df = parallel_apply(train, 'response_b')  # レスポンスBの統計を計算

In [ ]:
train = train.join(prompt_stats_df.add_suffix('_prompt'))  # プロンプトの統計をトレーニングデータに結合
train = train.join(response_a_stats_df.add_suffix('_response_a'))  # レスポンスAの統計をトレーニングデータに結合
train = train.join(response_b_stats_df.add_suffix('_response_b'))  # レスポンスBの統計をトレーニングデータに結合

In [ ]:
train.shape  # 結合後のトレーニングデータの形状を表示

In [ ]:
train.head()  # トレーニングデータの最初の数行を表示

In [ ]:
%%time

import time  # 時間計測のためのモジュールをインポート
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import log_loss
from scipy.stats import uniform, randint

# ターゲットを単一の列に変換し、カテゴリーベースにします
train['winner'] = (train['winner_model_a'] * 1 + train['winner_model_b'] * 2 + train['winner_tie'] * 3).astype(int)

# 特徴量とターゲットの定義
columns_to_remove = {'id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b', 
                     'winner_model_a', 'winner_model_b', 'winner_tie', 'winner'}

features = [col for col in train.columns if col not in columns_to_remove]  # 残す特徴量をリストアップ

X = train[features]  # 特徴量データ
y = train['winner'] - 1  # ターゲットデータ（0から始まるインデックスに変換）

In [ ]:
# 最適化関数の定義
def objective(trial):
    # トライアル内でのデータ分割
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # 最適化するモデルとハイパーパラメータの定義
    model_type = trial.suggest_categorical('model_type', ['XGBClassifier', 'LGBMClassifier', 'CatBoostClassifier'])
    n_estimators = trial.suggest_int('n_estimators', 100, 500)  # 決定木の数を指定
    max_depth = trial.suggest_int('max_depth', 2, 10)  # 木の深さを指定
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 0.1)  # 学習率を指定

    if model_type == 'XGBClassifier':
        model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, use_label_encoder=False, eval_metric='logloss', random_state=42)
    elif model_type == 'LGBMClassifier':
        model = LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, random_state=42)
    elif model_type == 'CatBoostClassifier':
        model = CatBoostClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, verbose=0, random_state=42)

    # モデルの訓練と評価
    model.fit(X_train, y_train)  # モデルを訓練
    y_val_pred = model.predict_proba(X_val)  # バリデーションデータで予測
    return log_loss(y_val, y_val_pred)  # ログ損失を返す

# Optuna最適化の実行
study = optuna.create_study(direction='minimize')  # 最小化方向でスタディを作成
study.optimize(objective, n_trials=5)  # トライアルを実行

print('最良のトライアル:', study.best_trial.params)  # 最良のパラメータを表示

# フルデータで最良モデルを訓練
best_params = study.best_trial.params  # 最良のパラメータを取得
model_type = best_params.pop('model_type')  # モデルタイプを取得し、辞書から削除

if model_type == 'XGBClassifier':
    final_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss', random_state=42)
elif model_type == 'LGBMClassifier':
    final_model = LGBMClassifier(**best_params, random_state=42)
elif model_type == 'CatBoostClassifier':
    final_model = CatBoostClassifier(**best_params, verbose=0, random_state=42)

final_model.fit(X, y)  # フルデータセットで訓練

In [ ]:
final_model  # 最終モデルを表示

In [ ]:
# 'prompt'および'response'列に対してparallel_applyを再度適用
prompt_stats_df_test = parallel_apply(test, 'prompt')  # テストデータのプロンプト統計を計算
response_a_stats_df_test = parallel_apply(test, 'response_a')  # テストデータのレスポンスA統計を計算
response_b_stats_df_test = parallel_apply(test, 'response_b')  # テストデータのレスポンスB統計を計算

In [ ]:
test = test.join(prompt_stats_df_test.add_suffix('_prompt'))  # プロンプトの統計をテストデータに結合
test = test.join(response_a_stats_df_test.add_suffix('_response_a'))  # レスポンスAの統計をテストデータに結合
test = test.join(response_b_stats_df_test.add_suffix('_response_b'))  # レスポンスBの統計をテストデータに結合

In [ ]:
test = test[features]  # テストデータから特徴量のみ抽出
test  # テストデータを表示

In [ ]:
train.head()  # トレーニングデータの最初の数行を表示

In [ ]:
test_predictions = final_model.predict_proba(test)  # テストデータに対する予測確率を計算

In [ ]:
test_predictions  # テストデータの予測確率を表示

In [ ]:
test_raw = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv', usecols=['id'])  # テストデータのIDを読み込む

In [ ]:
# 提出ファイルの準備
submission = pd.DataFrame({
    'id': test_raw['id'],  # ID列
    'winner_model_a': test_predictions[:, 0],  # モデルAの勝者予測
    'winner_model_b': test_predictions[:, 1],  # モデルBの勝者予測
    'winner_tie': test_predictions[:, 2]  # 引き分けの勝者予測
})

submission.head()  # 提出ファイルの最初の数行を表示

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)  # 提出ファイルをCSV形式で保存